In [ ]:
# ============================================
# 1. Import Libraries
# ============================================
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

In [ ]:
# ============================================
# 2. Setup GPU
# ============================================
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
# ============================================
# 3. Prepare Dataset
# ============================================
base_dir = 'dataset/'  # Ganti sesuai folder dataset kamu

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    shear_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


Found 5228 images belonging to 3 classes.
Found 5228 images belonging to 3 classes.


In [ ]:
# ============================================
# 4. Build Custom CNN
# ============================================
model_cnn = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(224,224,3)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')
])

model_cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_cnn.summary()


c:\Lang\Python\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# ============================================
# 5. Build Transfer Learning ResNet50 (Fine-tune)
# ============================================
# Load ResNet50 base
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224,224,3))
base_model.trainable = False  # freeze dulu

# Tambah classifier custom di atasnya
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(3, activation='softmax')(x)

model_resnet = Model(inputs=base_model.input, outputs=output)

# Compile awal
model_resnet.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_resnet.summary()



In [ ]:
# ============================================
# 6. Train Stage 1 (head saja)
# ============================================
history_resnet_stage1 = model_resnet.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

In [ ]:
# ============================================
# 7. Fine-tune: Unfreeze sebagian ResNet50
# ============================================
for layer in base_model.layers[:100]:  # Freeze dulu 100 layer pertama
    layer.trainable = False
for layer in base_model.layers[100:]:
    layer.trainable = True

# Re-compile dengan learning rate kecil
model_resnet.compile(optimizer=Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# ============================================
# 8. Train Stage 2 (fine-tuning)
# ============================================
history_resnet_stage2 = model_resnet.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

In [ ]:
# ============================================
# 9. Training Model CNN Buatan Sendiri
# ============================================
history_cnn = model_cnn.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

In [ ]:

# ============================================
# 10. Plot Accuracy & Loss
# ============================================
# Pilih mau plot CNN atau ResNet
history = history_resnet_stage2  # bisa ganti ke history_cnn kalau mau plot yang CNN

plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.title('Accuracy')
plt.legend()
plt.show()

plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Loss')
plt.legend()
plt.show()

In [ ]:
# ============================================
# 11. Evaluasi Akhir
# ============================================
val_loss, val_acc = model_resnet.evaluate(val_generator)
print(f"Akurasi Validation (ResNet50 Fine-tune): {val_acc:.2f}")

In [ ]:
# ============================================
# 12. Confusion Matrix + Classification Report
# ============================================
# Prediksi
Y_pred = model_resnet.predict(val_generator)
y_pred = np.argmax(Y_pred, axis=1)

# True labels
y_true = val_generator.classes

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

# Classification Report
target_names = list(val_generator.class_indices.keys())
print(classification_report(y_true, y_pred, target_names=target_names))